# LioNets: Turbofan Engine Degradation Simulation Dataset with Neural Networks

In this notebook, we present how LioNets can be applied in predictive models using time series data.

In [1]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
%matplotlib inline
from IPython.display import Image
from IPython.display import SVG
from IPython.display import display                               
from ipywidgets import interactive
import matplotlib.pyplot as plt
from collections import OrderedDict
import pandas as pd
import seaborn as sns
import numpy as np
import random
import re
from math import sqrt, exp, log
from sklearn.linear_model import Lasso, Ridge, RidgeCV, SGDRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score, balanced_accuracy_score, accuracy_score
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, TimeDistributed, RepeatVector, Flatten, Input, Dropout, LSTM, concatenate, Reshape, Conv1D, GlobalMaxPool1D
from keras.utils import plot_model

from lionets import LioNets
from nbeats_keras.model import NBeatsNet
from Interpretable_PCA import iPCA
from altruist.altruist import Altruist
from utilities.evaluation import Evaluation
from utilities.load_dataset import Load_Dataset

from lime.lime_text import LimeTextExplainer
from lime.lime_tabular import LimeTabularExplainer

Using TensorFlow backend.


First of all, we load and clean our data.

In [3]:
fm, feature_names = Load_Dataset.load_data_turbofan(False)

fm1_train = fm['FaultMode1']['df_train']
fm1_train_target = fm1_train['RUL'].values
fm1_test= fm['FaultMode1']['df_test']
fm1_test_target = fm1_test['RUL'].values

We are dropping some unecessary features.

In [4]:
LSTM_train = fm1_train.drop(columns=['t', 'os_1', 'os_2', 'os_3', 's_01', 's_05', 's_06', 's_10', 's_16', 's_18', 's_19', 's_22', 's_23', 's_24', 's_25', 's_26'])
LSTM_test = fm1_test.drop(columns=['t', 'os_1', 'os_2', 'os_3', 's_01', 's_05', 's_06', 's_10', 's_16', 's_18', 's_19', 's_22', 's_23', 's_24', 's_25', 's_26'])

We collect the different units, in order to the next steps to create time windows

In [5]:
train_units = set(LSTM_train['u'].values)
test_units = set(LSTM_test['u'].values)

We are scaling our data per feature

In [6]:
sensors = ['s_02', 's_03', 's_04', 's_07', 's_08', 's_09', 's_11', 's_12',
            's_13', 's_14', 's_15', 's_17', 's_20', 's_21']
scalers = {}
for column in sensors:
    scaler = MinMaxScaler(feature_range=(0,1))
    LSTM_train[column] = scaler.fit_transform(LSTM_train[column].values.reshape(-1,1))
    LSTM_test[column] = scaler.transform(LSTM_test[column].values.reshape(-1,1))
    scalers[column] = scaler

We create time windows with a specific size. In this example, we create time windows of 50 timesteps.

In [7]:
unit_scalers = {}
window = 50
temp_LSTM_x_train = []
LSTM_y_train = []
for unit in train_units:
    temp_unit = LSTM_train[LSTM_train['u']==unit].drop(columns=['u','RUL']).values
    temp_unit_RUL = LSTM_train[LSTM_train['u']==unit]['RUL'].values
    
    for i in range(len(temp_unit) - window + 1):#elekse edw an len temp_unit - window > 0
        temp_instance = []
        for j in range(window):
            temp_instance.append(temp_unit[i+j])
        temp_LSTM_x_train.append(np.array(temp_instance))
        LSTM_y_train.append(temp_unit_RUL[i+window-1])
LSTM_y_train = np.array(LSTM_y_train)
LSTM_x_train = np.array(temp_LSTM_x_train)

temp_LSTM_x_test = []
LSTM_y_test = []
for unit in test_units:
    temp_unit = LSTM_test[LSTM_test['u']==unit].drop(columns=['u','RUL']).values
    temp_unit_RUL = LSTM_test[LSTM_test['u']==unit]['RUL'].values
        
    for i in range(len(temp_unit) - window + 1):#elekse edw an len temp_unit - window > 0
        temp_instance = []
        for j in range(window):
            temp_instance.append(temp_unit[i+j])
        temp_LSTM_x_test.append(np.array(temp_instance))
        LSTM_y_test.append(temp_unit_RUL[i+window-1])
LSTM_y_test = np.array(LSTM_y_test)
LSTM_x_test = np.array(temp_LSTM_x_test)

We can check how many train, test instances we have. These are changing regarding the time window size.

In [8]:
LSTM_x_train.shape, LSTM_x_test.shape, LSTM_y_train.shape, LSTM_y_test.shape

((15731, 50, 14), (8255, 50, 14), (15731,), (8255,))

Moreover, we scale our target data, in order to train our models faster

In [9]:
temp_LSTM_y_train = [[i] for i in LSTM_y_train]
temp_LSTM_y_test = [[i] for i in LSTM_y_test]
target_scaler = MinMaxScaler()
target_scaler.fit(temp_LSTM_y_train)
temp_LSTM_y_train = target_scaler.transform(temp_LSTM_y_train)
temp_LSTM_y_test = target_scaler.transform(temp_LSTM_y_test)

In [10]:
LSTM_y_train[2],temp_LSTM_y_train[2]

(140, array([0.44871795]))

We need a rmse loss function too!

In [11]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

Now, we can build our predictor

In [12]:
feature_names = fm1_train.columns
encoder_input = Input(shape=(LSTM_x_train[0].shape))

encoder_x = LSTM(units=80, return_sequences=True, activation='tanh')(encoder_input)
encoder_x = Dropout(0.7)(encoder_x)
encoder_x = LSTM(units=40, return_sequences=False, activation='tanh')(encoder_x)

encoder_y = Conv1D(filters=40,kernel_size=3,activation='tanh')(encoder_input)
encoder_y = GlobalMaxPool1D()(encoder_y)

encoded = concatenate([encoder_x,encoder_y])
encoded = Dropout(0.7)(encoded)
encoded = Dense(80, activation='tanh')(encoded)#Relu and selu
encoded = Dropout(0.7)(encoded)
encoded = Dense(40, activation='tanh')(encoded)#Relu and selu
predictions = Dense(1, activation='sigmoid')(encoded)#Relu and selu
predictor = Model(encoder_input,predictions)

predictor.compile(optimizer="adam",loss=[root_mean_squared_error],metrics=['mae','mse'])
#print(predictor.summary())

checkpoint_name = 'TEDS_Predictor_RUL.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 2, save_best_only = True, mode ='auto')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Then, we train the predictor.

In [13]:
#predictor.fit(LSTM_x_train, temp_LSTM_y_train, epochs=500, batch_size=512, shuffle=True, validation_split=0.33, verbose=2, callbacks=[checkpoint])

We load our weights, and we measure the performance.

In [14]:
weights_file = 'weights/TEDS_Predictor_RUL.hdf5' # choose the best checkpoint few features
predictor.load_weights(weights_file) # load it
predictor.compile(optimizer="adam",loss=[root_mean_squared_error],metrics=['mae','mse'])

In [15]:
temp_pred = target_scaler.inverse_transform(predictor.predict(LSTM_x_train))
predictions = [i[0] for i in temp_pred]
print('Train:',mean_absolute_error(LSTM_y_train,predictions),mean_squared_error(LSTM_y_train,predictions),sqrt(mean_squared_error(LSTM_y_train,predictions)))
print(r2_score(LSTM_y_train,predictions))

Train: 11.486946112474099 321.77211100790623 17.938007442520092
0.9022143936099378


In [16]:
temp_pred = target_scaler.inverse_transform(predictor.predict(LSTM_x_test))
predictions = [i[0] for i in temp_pred]
print('Test:',mean_absolute_error(LSTM_y_test,predictions),mean_squared_error(LSTM_y_test,predictions),sqrt(mean_squared_error(LSTM_y_test,predictions)))
print(r2_score(LSTM_y_test,predictions))

Test: 22.958150270648034 1030.8807520057308 32.107331748460986
0.6217281552452669


Then, we have to extract the encoder from our predictor.

In [17]:
encoder = Model(input=predictor.input, output=[predictor.layers[-2].output])
encoder.trainable = False
encoder.compile(optimizer="adam",loss=[root_mean_squared_error],metrics=['mae','mse'])
#encoder.summary()

Now we are ready to extract for all instances, their encoded representation

In [18]:
encoded_LSTM_x_train = encoder.predict(LSTM_x_train)
encoded_LSTM_x_test = encoder.predict(LSTM_x_test)

And by that, we build the decoder.

In [19]:
encoded_input = Input(shape=(encoded_LSTM_x_train[0].shape))
decoded = Dense(120, activation='tanh')(encoded_input)
decoded = Dropout(0.5)(decoded)

decoded_y = RepeatVector(54)(decoded)
decoded_y = Conv1D(filters=50,kernel_size=5,activation='tanh')(decoded_y)

decoded_x = RepeatVector(50)(decoded)
decoded_x = LSTM(units=80, return_sequences=True, activation='tanh')(decoded_x)
decoded_x = Dropout(0.5)(decoded_x)
decoded_x = LSTM(units=50, return_sequences=True, activation='tanh')(decoded_x)

decoded = concatenate([decoded_x,decoded_y])
decoded = LSTM(50, return_sequences=True, activation='sigmoid')(decoded)
decoded = Dropout(0.5)(decoded)
decoded = LSTM(14, return_sequences=True, activation='sigmoid')(decoded)

decoder = Model(encoded_input,decoded)

decoder.compile(optimizer="adam",loss=[root_mean_squared_error],metrics=['mae','mse'])
#print(decoder.summary())

checkpoint_name = 'TEDS_Decoder_RUL.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 2, save_best_only = True, mode ='auto')

In [20]:
#decoder.fit(encoded_LSTM_x_train, LSTM_x_train, epochs=100, batch_size=2048, shuffle=True, validation_split=0.2, verbose=2, callbacks=[checkpoint])

In [21]:
weights_file = 'weights/TEDS_Decoder_RUL.hdf5' # choose the best checkpoint few features
decoder.load_weights(weights_file) # load it
decoder.compile(optimizer="adam",loss=[root_mean_squared_error],metrics=['mae','mse'])

In [22]:
#rmse, mae, mse
# decoder.evaluate(encoded_LSTM_x_train,LSTM_x_train)

In [23]:
# decoder.evaluate(encoded_LSTM_x_test,LSTM_x_test)

---
# Forecasters

Here we propose some forecasting options that are going to be implemented in the IML tool.

## Basic Neural Forecaster

In [24]:
window = 50
forecast_timesteps = 5

temp_fc_x_train = []
temp_fc_y_train = []
for unit in train_units:
    temp_unit = LSTM_train[LSTM_train['u']==unit].drop(columns=['u','RUL']).values
   
    for i in range(len(temp_unit) - window - forecast_timesteps + 1):#elekse edw an len temp_unit - window > 0
        temp_instance_x = []
        temp_instance_y = []
        for j in range(window):
            temp_instance_x.append(temp_unit[i+j])
        for z in range(forecast_timesteps):
            temp_instance_y.append(temp_unit[i+j+z+1])            
        temp_fc_x_train.append(np.array(temp_instance_x))
        temp_fc_y_train.append(np.array(temp_instance_y))       
fc_x_train = np.array(temp_fc_x_train)
fc_y_train = np.array(temp_fc_y_train)

temp_fc_x_test = []
temp_fc_y_test = []
for unit in test_units:
    temp_unit = LSTM_test[LSTM_test['u']==unit].drop(columns=['u','RUL']).values
        
    for i in range(len(temp_unit) - window - forecast_timesteps + 1):#elekse edw an len temp_unit - window > 0
        temp_instance_x = []
        temp_instance_y = []
        for j in range(window):
            temp_instance_x.append(temp_unit[i+j])
        for z in range(forecast_timesteps):
            temp_instance_y.append(temp_unit[i+j+z+1])            
        temp_fc_x_test.append(np.array(temp_instance_x))
        temp_fc_y_test.append(np.array(temp_instance_y))       
fc_x_test = np.array(temp_fc_x_test)
fc_y_test = np.array(temp_fc_y_test)

In [25]:
fc_x_train.shape, fc_x_test.shape, fc_y_train.shape, fc_y_test.shape

((15231, 50, 14), (7794, 50, 14), (15231, 5, 14), (7794, 5, 14))

In [26]:
forecast_input = Input(shape=(LSTM_x_train[0].shape))

forecast_x = LSTM(units=120, return_sequences=True, activation='tanh')(forecast_input)
forecast_x = Dropout(0.7)(forecast_x)
forecast_x = LSTM(units=50, return_sequences=True, activation='tanh')(forecast_x)
forecast_x = Conv1D(filters=50,kernel_size=46,activation='tanh')(forecast_x)

forecast_y = Conv1D(filters=50,kernel_size=46,activation='tanh')(forecast_input)

forecast = concatenate([forecast_y,forecast_x])
forecast = Dropout(0.7)(forecast)
forecast = LSTM(40, return_sequences=True, activation='relu')(forecast)#Relu and selu
forecast = Dropout(0.7)(forecast)
predictions = LSTM(14, return_sequences=True, activation='linear')(forecast)#Relu and selu
forecaster = Model(forecast_input,predictions)
forecaster.summary()
forecaster.compile(optimizer="adam", loss=[root_mean_squared_error],metrics=['mae','mse'])

checkpoint_name = 'weights/TEDS_Forecaster_RUL.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 2, save_best_only = True, mode ='auto')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 50, 14)       0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 50, 120)      64800       input_3[0][0]                    
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, 50, 120)      0           lstm_7[0][0]                     
__________________________________________________________________________________________________
lstm_8 (LSTM)                   (None, 50, 50)       34200       dropout_7[0][0]                  
__________________________________________________________________________________________________
conv1d_4 (

In [27]:
# forecaster.fit(fc_x_train, fc_y_train, epochs=250, batch_size=512, shuffle=True, validation_split=0.3, verbose=2, callbacks=[checkpoint])

In [28]:
weights_file = 'weights/TEDS_Forecaster_RUL.hdf5' # choose the best checkpoint few features
forecaster.load_weights(weights_file) # load it
forecaster.compile(optimizer="adam",loss=[root_mean_squared_error],metrics=['mae','mse'])

In [29]:
predictions = forecaster.predict(fc_x_train)
print('Train:',mean_absolute_error(fc_y_train.reshape(-1,70),predictions.reshape(-1,70)),mean_squared_error(fc_y_train.reshape(-1,70),predictions.reshape(-1,70)),sqrt(mean_squared_error(fc_y_train.reshape(-1,70),predictions.reshape(-1,70))))
print(r2_score(fc_y_train.reshape(-1,70),predictions.reshape(-1,70)))

Train: 0.05301874834831075 0.004864156445715008 0.06974350468477339
0.7355894053201308


In [30]:
predictions = forecaster.predict(fc_x_test)
print('Test:',mean_absolute_error(fc_y_test.reshape(-1,70),predictions.reshape(-1,70)),mean_squared_error(fc_y_test.reshape(-1,70),predictions.reshape(-1,70)),sqrt(mean_squared_error(fc_y_test.reshape(-1,70),predictions.reshape(-1,70))))
print(r2_score(fc_y_test.reshape(-1,70),predictions.reshape(-1,70)))

Test: 0.05329821010757238 0.004916158172408367 0.07011532052560529
0.5724395889274646


---
## N-Beats

In [31]:
# Definition of the model.
nbeats = NBeatsNet(input_dim = 14, backcast_length=50, forecast_length=5,
                  stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK), 
                  nb_blocks_per_stack=2,
                  thetas_dim=(4, 4), 
                  share_weights_in_stack=True, 
                  hidden_layer_units=64)

# Definition of the objective function and the optimizer.
nbeats.compile_model(loss='mae', learning_rate=1e-6)
 
# Train the model.
# nbeats.fit(fc_x_train, fc_y_train, verbose=1, validation_split=0.3, epochs=100, batch_size=128)

# Save the model for later.
# nbeats.save('weights/TEDS_NBeats_RUL.hdf5')

# # Load the model.
nbeats = NBeatsNet.load('weights/TEDS_NBeats_RUL.hdf5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_variable (InputLayer)     (None, 50, 14)       0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 50)           0           input_variable[0][0]             
__________________________________________________________________________________________________
0/0/generic/d1 (Dense)          (None, 64)           3264        lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
                                                                 lambda_4[0][0]                   
          

Instructions for updating:
Use tf.cast instead.


In [32]:
predictions = nbeats.predict(fc_x_train)
print('Train:',mean_absolute_error(fc_y_train.reshape(-1,70),predictions.reshape(-1,70)),mean_squared_error(fc_y_train.reshape(-1,70),predictions.reshape(-1,70)),sqrt(mean_squared_error(fc_y_train.reshape(-1,70),predictions.reshape(-1,70))))
print(r2_score(fc_y_train.reshape(-1,70),predictions.reshape(-1,70)))

Train: 0.1924214253250012 0.059098888844387874 0.24310263027040221
-3.211671873120382


In [33]:
predictions = nbeats.predict(fc_x_test)
print('Test:',mean_absolute_error(fc_y_test.reshape(-1,70),predictions.reshape(-1,70)),mean_squared_error(fc_y_test.reshape(-1,70),predictions.reshape(-1,70)),sqrt(mean_squared_error(fc_y_test.reshape(-1,70),predictions.reshape(-1,70))))
print(r2_score(fc_y_test.reshape(-1,70),predictions.reshape(-1,70)))

Test: 0.22186722158240046 0.07520374768411571 0.27423301712980463
-11.707985504125105


---
## XYZ7 Model

In [34]:
window = 50
forecast_steps = 5

rul_train, xyz7_x_train, xyz7_y_train, rul_temp = [],[],[],[]
for unit in train_units:
    temp_unit = LSTM_train[LSTM_train['u']==unit].drop(columns=['u','RUL']).values   
    for i in range(len(temp_unit) - window + 1): # elekse edw an len temp_unit - window > 0
        temp_instance = np.array(temp_unit[i:i+window])
        rul_temp.append(temp_instance)
        xyz7_x_train.append(temp_instance[:-forecast_steps])
        xyz7_y_train.append(temp_instance[-forecast_steps:])

rul_train = predictor.predict(np.array(rul_temp))
xyz7_x_train = np.array(xyz7_x_train)
xyz7_y_train = np.array(xyz7_y_train)

rul_test, xyz7_x_test, xyz7_y_test, rul_temp = [],[],[],[]
for unit in test_units:
    temp_unit = LSTM_test[LSTM_test['u']==unit].drop(columns=['u','RUL']).values   
    for i in range(len(temp_unit) - window + 1): # elekse edw an len temp_unit - window > 0
        temp_instance = np.array(temp_unit[i:i+window])
        rul_temp.append(temp_instance)
        xyz7_x_test.append(temp_instance[:-forecast_steps])
        xyz7_y_test.append(temp_instance[-forecast_steps:])

rul_test = predictor.predict(np.array(rul_temp))
xyz7_x_test = np.array(xyz7_x_test)
xyz7_y_test = np.array(xyz7_y_test)

In [35]:
rul_train.shape , xyz7_x_train.shape , xyz7_y_train.shape 

((15731, 1), (15731, 45, 14), (15731, 5, 14))

In [36]:
forecast_input = Input(shape=(xyz7_x_train[0].shape))
rul_input = Input(shape = (rul_train[0].shape))
rul = RepeatVector(5)(rul_input)

forecast_x = LSTM(units=120, return_sequences=True, activation='tanh')(forecast_input)
forecast_x = Dropout(0.7)(forecast_x)
forecast_x = LSTM(units=50, return_sequences=True, activation='tanh')(forecast_x)
forecast_x = Conv1D(filters=50,kernel_size=41,activation='tanh')(forecast_x)
forecast_x = concatenate([forecast_x, rul])


forecast_y = Conv1D(filters=50,kernel_size=41,activation='tanh')(forecast_input)
forecast_y = concatenate([forecast_y, rul])


forecast = concatenate([forecast_y, forecast_x, rul])
forecast = Dropout(0.7)(forecast)
forecast = LSTM(40, return_sequences=True, activation='relu')(forecast)#Relu and selu
forecast = concatenate([forecast, rul])
forecast = Dropout(0.7)(forecast)
predictions = LSTM(14, return_sequences=True, activation='linear')(forecast)#Relu and selu

xyz7_model = Model([forecast_input, rul_input],predictions)
opt = keras.optimizers.Adam(lr=0.001)
xyz7_model.compile(optimizer=opt, loss=[root_mean_squared_error],metrics=['mae','mse'])
xyz7_model.summary()

checkpoint_name = 'weights/TEDS_XYZ7_RUL.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 2, save_best_only = True, mode ='auto')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 45, 14)       0                                            
__________________________________________________________________________________________________
lstm_11 (LSTM)                  (None, 45, 120)      64800       input_4[0][0]                    
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 45, 120)      0           lstm_11[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
lstm_12 (L

In [37]:
# xyz7_model.fit([xyz7_x_train,rul_train], xyz7_y_train, epochs=250, batch_size=512, shuffle=True, validation_split=0.3, verbose=2, callbacks=[checkpoint])

In [38]:
weights_file = 'weights/TEDS_XYZ7_RUL.hdf5' # choose the best checkpoint few features
xyz7_model.load_weights(weights_file) # load it
xyz7_model.compile(optimizer=opt,loss=[root_mean_squared_error],metrics=['mae','mse'])

---
## IML-Tool 

In [39]:
lionet = LioNets(predictor, decoder, encoder, LSTM_x_train)

iml_methods = {'LioNets':lionet} # {'Lime':lime}

nn_forecasters = {'forecaster':forecaster, 'nbeats':nbeats, 'xyz7_model':xyz7_model}

In [40]:
from Interactive_IML_Tool import interactive_iml_tool

In [41]:
iml_tool = interactive_iml_tool(predictor, iml_methods, nn_forecasters, LSTM_x_train, LSTM_y_train, sensors, target_scaler)

In [49]:
temp_instance = LSTM_x_train[112].copy()
iml_tool.load_instance(temp_instance)

Output()

## LioNets & iPCA Experiments
Having everything setted up, we are now ready to try our methodology. We first initialize LioNets and iPCA. LioNets requires a predictor (the regressor itself), an encoder (extracted from the predictor), a decoder, as well as some data (for best results the training data, in order to push the neighbourhood generation through known distribution for the network). For the iPCA we use the neighbourhood generator of LioNets in order to build the local linear model. 

In [43]:
lionet = LioNets(predictor, decoder, encoder, LSTM_x_train, double_detector=False)
transparent_model = Ridge(alpha=200,fit_intercept=True,random_state=0)

In [44]:
random.seed(2000)
train = np.array(random.sample(LSTM_x_train.tolist(),200))
valid = np.array(random.sample(LSTM_x_test.tolist(),200))
train.shape, valid.shape

((200, 50, 14), (200, 50, 14))

Let's calculate the fidelity of Lime and LioNets

In [45]:
def lime_predict(instance):
    t_instance = np.array([instance]).reshape((len(instance),50,14))
    a = predictor.predict(t_instance)
    a = np.array([i[0] for i in a]) 
    return a

explainer = LimeTabularExplainer(training_data=train.reshape(((len(train), 700))), 
                 discretize_continuous=False,
                 mode="regression")

In [46]:
def fi_lime(instance):
    t_instance = instance.reshape((700))
    explanation,_,_,_ = explainer.explain_instance(t_instance, predict_fn=lime_predict, num_features=700)
    local_pred = explanation.local_pred[0]
    return local_pred #This is because lime interprets class with label 1
def fi_lionets(instance):
    weights, res, loc_res = lionet.explain_instance(instance,3000,transparent_model)
    return loc_res

In [47]:
evaluator = Evaluation(predictor.predict,None,lambda x: x,True)

In [48]:
fidelity = evaluator.fidelity(train, [fi_lime, fi_lionets], class_n=0)
print("Train:")
print('  Lime fidelity:', fidelity[0][0])
print('  LioNets fidelity:', fidelity[1][0])
fidelity = evaluator.fidelity(valid, [fi_lime, fi_lionets], class_n=0)
print("Valid:")
print('  Lime fidelity:', fidelity[0][0])
print('  LioNets fidelity:', fidelity[1][0])

KeyboardInterrupt: 

Altruist Score:

In [ ]:
robustness = evaluator.robustness(train,[fi_ipca, fi_GxI, fi_LRP, fi_lionets],None, [700,[50,14]])
print("Train")
print('  iPCA Robustness:', robustness[0])
print('  GxI Robustness:', robustness[1])
print('  LRP Robustness:', robustness[2])
print('  LioNets Robustness:', robustness[3])
robustness = evaluator.robustness(valid,[fi_ipca, fi_GxI, fi_LRP, fi_lionets],None, [700,[50,14]])
print("Valid:")
print('  iPCA Robustness:', robustness[0])
print('  GxI Robustness:', robustness[1])
print('  LRP Robustness:', robustness[2])
print('  LioNets Robustness:', robustness[3])

In [ ]:
def fi_lime(instance):
    explanation, _, _, _ = explainer.explain_instance(instance, predict_fn=lime_predict, num_features=700)
    weights = OrderedDict(explanation.as_list())
    lime_w = dict(sorted(zip(list([int(wk) for wk in weights.keys()]), list(weights.values()))))
    lweights = np.array([lime_w[o] for o in lime_w.keys()]).reshape((50,14))
    weights = []
    for i in range(14):
        weights.append(lweights[:,i:i+1].mean())
    return np.array(lweights)
def fi_lionets(instance):
    weights, res, loc_res = lionet.explain_instance(instance.reshape((50,14)),3000,transparent_model)
    lweights = weights.reshape((50,14))
    weights = []
    for i in range(14):
        weights.append(lweights[:,i:i+1].mean())
    return np.array(lweights)

In [ ]:
print("*Please let it run, it will take time probably*")
fi_names = {fi_lime:'Lime',fi_lionets:'LioNets'}
fis = [fi_lime,fi_lionets]
fis_scores = []
for i in fis:
    fis_scores.append([])
count = 0
feature_names = [i for i in range(14)]
X_t = np.array([inst.reshape((700)) for inst in train])
for instance in X_t:
    if (count + 1) % 50 == 0:
        print(count+1,"/",len(train),"..",end=", ")
    #print(len(instance))
    count = count + 1
    altruistino = Altruist(predictor, X_t, fis, feature_names, None, True,[50,14])
    untruthful_features = altruistino.find_untruthful_features(instance)
    for i in range(len(untruthful_features[0])):
        fis_scores[i].append(len(untruthful_features[0][i]))
count = 0
print()
print("Train:")
for fis_score in fis_scores:
    fi = fis[count]
    count = count + 1
    print(' ',fi_names[fi],np.array(fis_score).mean())
fi_matrix = np.array(fis_scores)
count = 0
fi_all = []
for instance in X_t:
    fi_all.append(fi_matrix[:,count].min())
    count = count + 1
print("Altogether:",np.array(fi_all).mean())

In [ ]:
print("*Please let it run, it will take time probably*")
fi_names = {fi_lime:'Lime',fi_lionets:'LioNets'}
fis = [fi_lime,fi_lionets]
fis_scores = []
for i in fis:
    fis_scores.append([])
count = 0
feature_names = [i for i in range(14)]
X_v = np.array([inst.reshape((700)) for inst in valid])
for instance in X_v[:2]:
    if (count + 1) % 50 == 0:
        print(count+1,"/",len(train),"..",end=", ")
    #print(len(instance))
    count = count + 1
    altruistino = Altruist(predictor, X_t, fis, feature_names, None, True,[50,14])
    untruthful_features = altruistino.find_untruthful_features(instance)
    for i in range(len(untruthful_features[0])):
        fis_scores[i].append(len(untruthful_features[0][i]))
count = 0
print()
print("Valid:")
for fis_score in fis_scores:
    fi = fis[count]
    count = count + 1
    print(' ',fi_names[fi],np.array(fis_score).mean())
fi_matrix = np.array(fis_scores)
count = 0
fi_all = []
for instance in X_t:
    fi_all.append(fi_matrix[:,count].min())
    count = count + 1
print("Altogether:",np.array(fi_all).mean())

# Qualitative Experiments
For a given example, its prediction and its interpretation, we would like to enhance it's RUL value.

In [ ]:
temp_instance = LSTM_x_train[112].copy()
model = Ridge(alpha=200,fit_intercept=True,random_state=0)
weights, real_prediction, local_prediction = lionet.explain_instance(temp_instance,3000,model)

In [ ]:
explanation, _, _, _ = explainer.explain_instance(temp_instance.flatten(), predict_fn=lime_predict, num_features=700)
weights = OrderedDict(explanation.as_list())
lime_w = dict(sorted(zip(list([int(wk) for wk in weights.keys()]), list(weights.values()))))
lweights = np.array([lime_w[o] for o in lime_w.keys()])

In [ ]:
"Real prediction: " + str(real_prediction)[:7] + ", Local prediction: " + str(local_prediction)[:7]

In [ ]:
target_scaler.inverse_transform(np.array([[0.08028],[0.08079]]))

We would like to increase the RUL!

From LioNets we acquired the weights of each sensor's measurements. Then we extract some statistics

In [ ]:
sensors_all = {}
count = 0
for j in range(50):
    count2 = 0
    for i in sensors:
        sensors_all.setdefault(i,[]).append([j, weights[count+count2], temp_instance[j][count2],
                                             weights[count+count2]*temp_instance[j][count2]])
        count2 = count2 + 1
    count = count + 14
sensors_std = []
sensors_mean = []
sensors_max = []
sensors_min = []
for i in sensors_all:
    naa = np.array(sensors_all[i])[:,3]
    sensors_std.append(naa.std())
    sensors_mean.append(naa.mean())
    sensors_max.append(naa.max())
    sensors_min.append(naa.min())
    #print(i, naa.mean(), naa.std(), naa.max(), naa.min())
statistics = pd.DataFrame({"Sensor": list(sensors), "Mean": list(sensors_mean), "STD": list(sensors_std), 
                           "Max": list(sensors_max), "Min": list(sensors_min), 
                           "Max-Min": np.array(sensors_max) + np.array(sensors_min)})

In [ ]:
to_vis = [i[2:] for i in sensors]
fig, axs = plt.subplots(1, 3, figsize=(14, 4), dpi=200)
sns.barplot(to_vis,sensors_mean,ax=axs[0])
axs[0].set_title('Mean')
sns.barplot(to_vis,sensors_std,ax=axs[1])
axs[1].set_title('STD')
sns.barplot(to_vis,sensors_max,ax=axs[2])
sns.barplot(to_vis,sensors_min,ax=axs[2])
axs[2].set_title('Max and Min')
fig.suptitle('Sensor Importance Statistics')
plt.show()

def plot_sensor(sens_i=1):
    plt.figure(figsize=(14, 4), dpi=200, facecolor='w', edgecolor='k')
    plt.subplot(131)
    sns.lineplot(np.array(sensors_all['s_02'])[:,0],np.array(sensors_all[sensors[sens_i-1]])[:,1])
    plt.hlines(y=np.array(sensors_all[sensors[sens_i-1]])[:,1].mean(), xmin=0, xmax=50, label='mean')
    plt.title(str("Sensor\'s " + sensors[sens_i-1] + " influence"))
    plt.subplot(132)
    sns.lineplot(np.array(sensors_all['s_02'])[:,0],np.array(sensors_all[sensors[sens_i-1]])[:,2],color='g')
    plt.hlines(y=np.array(sensors_all[sensors[sens_i-1]])[:,2].mean(), xmin=0, xmax=50, label='mean')
    plt.title(str("Sensor\'s " + sensors[sens_i-1] + " value"))
    plt.subplot(133)
    sns.lineplot(np.array(sensors_all['s_02'])[:,0],np.array(sensors_all[sensors[sens_i-1]])[:,3],color='r')
    plt.hlines(y=np.array(sensors_all[sensors[sens_i-1]])[:,3].mean(), xmin=0, xmax=50, label='mean')
    plt.title(str("Sensor\'s " + sensors[sens_i-1] + " influence * value"))
    plt.show()
inter=interactive(plot_sensor 
   , sens_i=(1,14))
display(inter)

Let's modify the measurements from the s_12 sensor (number 8), which is is influencing negatively the RUL:

In [ ]:
sens = 3-1
for i in range(35,50):
    if weights.reshape(50,14)[i:i+1,sens:sens+1][0] < -0.0005:
        temp_instance[i:i+1,sens:sens+1][0]=temp_instance[i:i+1,sens:sens+1][0]-0.15

Now let's test the modified instance

In [ ]:
weights, real_prediction, local_prediction = lionet.explain_instance(temp_instance,3000,model)
weights = weights * temp_instance.reshape(700)

In [ ]:
"Real prediction: " + str(real_prediction)[:7] + ", Local prediction: " + str(local_prediction)[:7]

In [ ]:
target_scaler.inverse_transform(np.array([[0.10759],[0.10512]]))

We managed to increase the RUL, a little. (Try and modify s_07 as well with the 12th sensor). How much does the RUL increased?

In [ ]:
sensors_all = {}
count = 0
for j in range(50):
    count2 = 0
    for i in sensors:
        sensors_all.setdefault(i,[]).append([j, weights[count+count2], temp_instance[j][count2],
                                             weights[count+count2]*temp_instance[j][count2]])
        count2 = count2 + 1
    count = count + 14
sensors_std = []
sensors_mean = []
sensors_max = []
sensors_min = []
for i in sensors_all:
    naa = np.array(sensors_all[i])[:,3]
    sensors_std.append(naa.std())
    sensors_mean.append(naa.mean())
    sensors_max.append(naa.max())
    sensors_min.append(naa.min())
    #print(i, naa.mean(), naa.std(), naa.max(), naa.min())
statistics = pd.DataFrame({"Sensor": list(sensors), "Mean": list(sensors_mean), "STD": list(sensors_std), 
                           "Max": list(sensors_max), "Min": list(sensors_min), 
                           "Max-Min": np.array(sensors_max) + np.array(sensors_min)})

In [ ]:
to_vis = [i[2:] for i in sensors]
fig, axs = plt.subplots(1, 3, figsize=(14, 4), dpi=200)
sns.barplot(to_vis,sensors_mean,ax=axs[0])
axs[0].set_title('Mean')
sns.barplot(to_vis,sensors_std,ax=axs[1])
axs[1].set_title('STD')
sns.barplot(to_vis,sensors_max,ax=axs[2])
sns.barplot(to_vis,sensors_min,ax=axs[2])
axs[2].set_title('Max and Min')
fig.suptitle('Sensor Importance Statistics')
plt.show()

def plot_sensor(sens_i=1):
    plt.figure(figsize=(14, 4), dpi=200, facecolor='w', edgecolor='k')
    plt.subplot(131)
    sns.lineplot(np.array(sensors_all['s_02'])[:,0],np.array(sensors_all[sensors[sens_i-1]])[:,1])
    plt.hlines(y=np.array(sensors_all[sensors[sens_i-1]])[:,1].mean(), xmin=0, xmax=50, label='mean')
    plt.title(str("Sensor\'s " + sensors[sens_i-1] + " influence"))
    plt.subplot(132)
    sns.lineplot(np.array(sensors_all['s_02'])[:,0],np.array(sensors_all[sensors[sens_i-1]])[:,2],color='g')
    plt.hlines(y=np.array(sensors_all[sensors[sens_i-1]])[:,2].mean(), xmin=0, xmax=50, label='mean')
    plt.title(str("Sensor\'s " + sensors[sens_i-1] + " value"))
    plt.subplot(133)
    sns.lineplot(np.array(sensors_all['s_02'])[:,0],np.array(sensors_all[sensors[sens_i-1]])[:,3],color='r')
    plt.hlines(y=np.array(sensors_all[sensors[sens_i-1]])[:,3].mean(), xmin=0, xmax=50, label='mean')
    plt.title(str("Sensor\'s " + sensors[sens_i-1] + " influence * value"))
    plt.show()
inter=interactive(plot_sensor 
   , sens_i=(1,14))
display(inter)

Try for another instance, and play with the plots :) Thanks for using LioNets.

For any question contact us at GitHub repo: https://github.com/intelligence-csd-auth-gr/LionLearn.git

or at our lab's website: https://intelligence.csd.auth.gr

In [ ]:
# Plotting the figures 
#         expected = self.expected_sens_stats if mod==9 and rd_btn_xyz7 else [None]*4
            
        if iml_method == 'LioNets':
            fig, axs = plt.subplots(1, 3, figsize=(18, 4), dpi=200)
            if mod==9 and rd_btn_xyz7:
                self.plot_bar_chart(axs[0], self.original_sens_stats[0], self.mod_sens_stats[0], self.expected_sens_stats[0])
                self.plot_bar_chart(axs[1], self.original_sens_stats[1], self.mod_sens_stats[1], self.expected_sens_stats[1])
                self.plot_bar_chart(axs[2], self.original_sens_stats[2], self.mod_sens_stats[2], self.expected_sens_stats[2])
                self.plot_bar_chart(axs[2], self.original_sens_stats[3], self.mod_sens_stats[3], self.expected_sens_stats[3])
            else:
                self.plot_bar_chart(axs[0], self.original_sens_stats[0], self.mod_sens_stats[0])
                self.plot_bar_chart(axs[1], self.original_sens_stats[1], self.mod_sens_stats[1])
                self.plot_bar_chart(axs[2], self.original_sens_stats[2], self.mod_sens_stats[2])
                self.plot_bar_chart(axs[2], self.original_sens_stats[3], self.mod_sens_stats[3])
            axs[0].set_title('Mean')
            axs[1].set_title('STD')
            axs[2].set_title('Max and Min')
            plt.show()
            
        else:
            fig, axs  = plt.subplots(1,3,figsize=(18, 4))
            self.plot_bar_chart(axs[1], self.original_sens_stats, self.mod_sens_stats, self.expected_sens_stats)
            axs[1].set_title('Sensor Importance ')
            axs[0].axis('off')
            
def plot_bar_chart(self, axs, a, b, c=[]):
        to_vis = [i[2:] for i in self.sensors]
        ind = np.arange(len(to_vis))
        w = 0.5
        
        if c==[]:
            colors = ['C0', 'C1']
            labels = ('Οriginal','Modded')
            for x, ha, hb in zip(ind , a, b):
                for i, (s, c) in enumerate(sorted(zip([ha, hb], colors))):
                    axs.bar(x, s, color=c, zorder=-i)
            axs.set_xticklabels(to_vis)
        else:
            colors = ['C0', 'C1', 'C2']
            labels = ('Οriginal','Modded','Expected')
            for x, za, zb, zc in zip(ind, a, b, c):
                for i, (s, c) in enumerate(sorted(zip([za, zb, zc], colors))):
                    axs.bar(x, s, color=c, zorder=-i)
            axs.set_xticklabels(to_vis)
        return axs